# アクセスログの分析
学生番号:21060009  
名前:坂島悠太  
## 目的
目的
## 背景
背景
## 分析
### 前処理
アパッチのアクセスログをcsv形式に変換し、access.csvとして保存します。

In [1]:
# データ前処理
import csv
import re

def parse_log_line(line):
    pattern = r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}) - - \[(.*?)\] "(.*?)" (\d{3}) (\d+) "(.*?)" "(.*?)"'
    match = re.match(pattern, line)
    if match:
        return match.groups()
    return None

def convert_log_to_csv(log_file_path, csv_file_path):
    with open(log_file_path, 'r') as log_file, open(csv_file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['IP', 'DateTime', 'Request', 'Status', 'Bytes', 'Referer', 'UserAgent'])
        for line in log_file:
            parsed_line = parse_log_line(line)
            if parsed_line:
                writer.writerow(parsed_line)

# 使用例
convert_log_to_csv('./data/access.log', './data/access.csv')


### データの読み込み
先ほど作成した./data/access.csvをdataframeに読み込みます

In [3]:
import pandas as pd

# CSVファイルのパス
csv_file_path = './data/access.csv'

# CSVファイルを読み込む
df = pd.read_csv(csv_file_path)

# DataFrameを表示する
print(df.head(5))


              IP                    DateTime  \
0   54.36.149.41  22/Jan/2019:03:56:14 +0330   
1    31.56.96.51  22/Jan/2019:03:56:16 +0330   
2    31.56.96.51  22/Jan/2019:03:56:16 +0330   
3  40.77.167.129  22/Jan/2019:03:56:17 +0330   
4    91.99.72.15  22/Jan/2019:03:56:17 +0330   

                                             Request  Status  Bytes  \
0  GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%D...     200  30577   
1     GET /image/60844/productModel/200x200 HTTP/1.1     200   5667   
2     GET /image/61474/productModel/200x200 HTTP/1.1     200   5379   
3     GET /image/14925/productModel/100x100 HTTP/1.1     200   1696   
4  GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8...     200  41483   

                               Referer  \
0                                    -   
1  https://www.zanbil.ir/m/filter/b113   
2  https://www.zanbil.ir/m/filter/b113   
3                                    -   
4                                    -   

                               

## 流入経路
ユーザーがどういった経路でこのサイトにたどり着いたのかを示すリファラのランキングです